# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

# NLP
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# sklearn
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, f1_score
from sklearn.externals import joblib
# from sklearn.metrics import confusion_matrix
# 
# from sklearn.base import BaseEstimator, TransformerMixin


[nltk_data] Downloading package punkt to /Users/stingl/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/stingl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/stingl/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///messages.db')
connection = engine.connect()
df = pd.read_sql_table("messages", con=connection)
X = df.iloc[:, 1]
Y = df.iloc[:, 4:] 

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('mcfl', MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators = 10)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
def show_metrics(Y_test, Y_pred):
    """
    Print f1-score, precision and recall for each category
    
    Parameters:
    -----------
    Y_test: True values,
    Y_pred: Predicted values
    """
    for cat in range(0, Y_test.shape[1]):
        cf=classification_report(Y_test.iloc[:, cat], pd.DataFrame(Y_pred).iloc[:, cat], output_dict=True).get('1')
        if cf is not None:
            print(f"Category:{cat: 3d}, f1-score: {cf['f1-score'] :5.3f}, precision: {cf['precision']: 5.3f}, recall: {cf['recall']: 5.3f}")
        else:
            print(f"Caterogy:{cat: 3d} doesn't have value 1")
                  
Y_pred = pipeline.predict(X_test)                  
show_metrics(Y_test, Y_pred)

Category:  0, f1-score: 0.877, precision:  0.841, recall:  0.917
Category:  1, f1-score: 0.548, precision:  0.761, recall:  0.428
Category:  2, f1-score: 0.000, precision:  0.000, recall:  0.000
Category:  3, f1-score: 0.676, precision:  0.744, recall:  0.620
Category:  4, f1-score: 0.127, precision:  0.582, recall:  0.071
Category:  5, f1-score: 0.212, precision:  0.684, recall:  0.126
Category:  6, f1-score: 0.079, precision:  0.714, recall:  0.042
Category:  7, f1-score: 0.013, precision:  0.333, recall:  0.006
Category:  8, f1-score: 0.133, precision:  0.588, recall:  0.075
Caterogy:  9 doesn't have value 1
Category: 10, f1-score: 0.466, precision:  0.854, recall:  0.320
Category: 11, f1-score: 0.563, precision:  0.831, recall:  0.426
Category: 12, f1-score: 0.467, precision:  0.812, recall:  0.328
Category: 13, f1-score: 0.338, precision:  0.730, recall:  0.220
Category: 14, f1-score: 0.079, precision:  0.800, recall:  0.042
Category: 15, f1-score: 0.022, precision:  1.000, recall

/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [7]:
print(classification_report(Y_test, Y_pred))

/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


              precision    recall  f1-score   support

           0       0.84      0.92      0.88      5941
           1       0.76      0.43      0.55      1333
           2       0.00      0.00      0.00        34
           3       0.74      0.62      0.68      3286
           4       0.58      0.07      0.13       644
           5       0.68      0.13      0.21       414
           6       0.71      0.04      0.08       239
           7       0.33      0.01      0.01       156
           8       0.59      0.07      0.13       267
           9       0.00      0.00      0.00         0
          10       0.85      0.32      0.47       512
          11       0.83      0.43      0.56       878
          12       0.81      0.33      0.47       714
          13       0.73      0.22      0.34       123
          14       0.80      0.04      0.08       192
          15       1.00      0.01      0.02        88
          16       0.67      0.05      0.10       259
          17       0.79    

/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x1a182ae200>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('mcfl',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
               oob_score=False, random_state=Non

In [9]:
parameters = {
#     'vect__binary': [True, False], 
    'mcfl__estimator__max_depth': [20, 50, 100],
    'mcfl__estimator__min_samples_split': [2, 5, 20],
    'mcfl__estimator__n_estimators': [5, 10, 20]
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10, cv=3, n_jobs=12, scoring='f1_micro')

cv.fit(X_train, Y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:   40.7s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:   45.5s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:  1.5min
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:  2.3min
[Parallel(n_jobs=12)]: Done  37 tasks      | elapsed:  3.4min
[Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:  4.2min
[Parallel(n_jobs=12)]: Done  67 out of  81 | elapsed:  6.4min remaining:  1.3min
[Parallel(n_jobs=12)]: Done  76 out of  81 | elapsed:  7.4min remaining:   29.2s
[Parallel(n_jobs=12)]: Done  81 out of  81 | elapsed:  7.7min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=12,
       param_grid={'mcfl__estimator__max_depth': [20, 50, 100], 'mcfl__estimator__min_samples_split': [2, 5, 20], 'mcfl__estimator__n_estimators': [5, 10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=10)

In [10]:
cv.best_params_

{'mcfl__estimator__max_depth': 100,
 'mcfl__estimator__min_samples_split': 20,
 'mcfl__estimator__n_estimators': 20}

In [11]:
cv.best_score_

0.6037436259084179

In [12]:
cv.score(X_test, Y_test)

0.588597450835729

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
Y_pred = cv.predict(X_test)

show_metrics(Y_test, Y_pred)

Category:  0, f1-score: 0.880, precision:  0.797, recall:  0.983
Category:  1, f1-score: 0.539, precision:  0.858, recall:  0.393
Category:  2, f1-score: 0.000, precision:  0.000, recall:  0.000
Category:  3, f1-score: 0.698, precision:  0.776, recall:  0.634
Category:  4, f1-score: 0.082, precision:  0.667, recall:  0.043
Category:  5, f1-score: 0.100, precision:  0.786, recall:  0.053
Category:  6, f1-score: 0.025, precision:  0.750, recall:  0.013
Category:  7, f1-score: 0.013, precision:  0.333, recall:  0.006
Category:  8, f1-score: 0.071, precision:  0.714, recall:  0.037
Caterogy:  9 doesn't have value 1
Category: 10, f1-score: 0.331, precision:  0.897, recall:  0.203
Category: 11, f1-score: 0.421, precision:  0.854, recall:  0.279
Category: 12, f1-score: 0.327, precision:  0.894, recall:  0.200
Category: 13, f1-score: 0.078, precision:  0.833, recall:  0.041
Category: 14, f1-score: 0.021, precision:  1.000, recall:  0.010
Category: 15, f1-score: 0.000, precision:  0.000, recall

/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [14]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.80      0.98      0.88      5941
           1       0.86      0.39      0.54      1333
           2       0.00      0.00      0.00        34
           3       0.78      0.63      0.70      3286
           4       0.67      0.04      0.08       644
           5       0.79      0.05      0.10       414
           6       0.75      0.01      0.02       239
           7       0.33      0.01      0.01       156
           8       0.71      0.04      0.07       267
           9       0.00      0.00      0.00         0
          10       0.90      0.20      0.33       512
          11       0.85      0.28      0.42       878
          12       0.89      0.20      0.33       714
          13       0.83      0.04      0.08       123
          14       1.00      0.01      0.02       192
          15       0.00      0.00      0.00        88
          16       0.50      0.02      0.04       259
          17       0.91    

/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

# Drop columns with only 1 value
"child_alone" and maybe other columns have only one value. So drop them because they may cause problems with some Classificators.

In [15]:
def drop_single_value_columns(data):
    """
    Drop any column which doesn't contain two diffent values
    
    Parameters:
    -----------
    data: DataFrame
    
    Return value:
    -------------
    DataFrame with column(s) removed
    """
    
    data.columns[data.nunique(axis=0) != 2]
    data.drop(columns=data.columns[data.nunique(axis=0) != 2], inplace=True)
    return data

In [16]:
X = df.iloc[:, 1]
Y = drop_single_value_columns(df.iloc[:, 4:]) 


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('mcfl', MultiOutputClassifier(estimator=SVC()))
])

parameters = {
#     'mcfl__estimator__kernel' : ['linear', 'rbf', 'sigmoid'],
    'mcfl__estimator__kernel' : ['linear'],
#     'mcfl__estimator__degree' : [3, 4, 5, 6],
#     'mcfl__estimator__kernel' : ['poly'],
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10, cv=3, n_jobs=12, scoring='f1_micro')

cv.fit(X_train, Y_train)

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=12)]: Done   3 out of   3 | elapsed:  6.7min remaining:    0.0s
[Parallel(n_jobs=12)]: Done   3 out of   3 | elapsed:  6.7min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ity=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=12,
       param_grid={'mcfl__estimator__kernel': ['linear']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=10)

In [17]:
Y_pred = cv.predict(X_test)
print(cv.best_params_)
print(cv.best_score_)
print(classification_report(Y_test, Y_pred)) 

{'mcfl__estimator__kernel': 'linear'}
0.6772289922751838
              precision    recall  f1-score   support

           0       0.86      0.93      0.89      5941
           1       0.78      0.58      0.67      1333
           2       0.00      0.00      0.00        34
           3       0.75      0.69      0.72      3286
           4       0.65      0.20      0.31       644
           5       0.75      0.26      0.39       414
           6       0.79      0.13      0.22       239
           7       0.00      0.00      0.00       156
           8       0.68      0.25      0.37       267
           9       0.75      0.66      0.70       512
          10       0.79      0.76      0.78       878
          11       0.80      0.55      0.65       714
          12       0.76      0.45      0.56       123
          13       0.81      0.13      0.22       192
          14       0.57      0.18      0.28        88
          15       0.62      0.19      0.29       259
          16       0.78 

/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [18]:
X = df.iloc[:, 1]
Y = df.iloc[:, 4:].drop(columns='child_alone')

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('mcfl', MultiOutputClassifier(estimator=SVC()))
])

parameters = {
#     'mcfl__estimator__kernel' : ['linear', 'rbf', 'sigmoid'],
    'mcfl__estimator__degree' : [2, 3, 4, 5, 6],
    'mcfl__estimator__kernel' : ['poly'],
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10, cv=3, n_jobs=12, scoring='f1_micro')

cv.fit(X_train, Y_train)
Y_pred = cv.predict(X_test)
print(cv.best_params_)
print(cv.best_score_)
print(classification_report(Y_test, Y_pred)) 

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   2 out of  15 | elapsed:  9.5min remaining: 62.0min
[Parallel(n_jobs=12)]: Done   4 out of  15 | elapsed:  9.6min remaining: 26.3min
[Parallel(n_jobs=12)]: Done   6 out of  15 | elapsed:  9.7min remaining: 14.5min
[Parallel(n_jobs=12)]: Done   8 out of  15 | elapsed:  9.7min remaining:  8.5min
[Parallel(n_jobs=12)]: Done  10 out of  15 | elapsed:  9.7min remaining:  4.8min
[Parallel(n_jobs=12)]: Done  12 out of  15 | elapsed:  9.7min remaining:  2.4min
[Parallel(n_jobs=12)]: Done  15 out of  15 | elapsed: 13.9min finished
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/stingl/anaconda3/envs/nd

/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarn

{'mcfl__estimator__degree': 2, 'mcfl__estimator__kernel': 'poly'}
0.3669025064350609
              precision    recall  f1-score   support

           0       0.76      1.00      0.86      5941
           1       0.00      0.00      0.00      1333
           2       0.00      0.00      0.00        34
           3       0.00      0.00      0.00      3286
           4       0.00      0.00      0.00       644
           5       0.00      0.00      0.00       414
           6       0.00      0.00      0.00       239
           7       0.00      0.00      0.00       156
           8       0.00      0.00      0.00       267
           9       0.00      0.00      0.00       512
          10       0.00      0.00      0.00       878
          11       0.00      0.00      0.00       714
          12       0.00      0.00      0.00       123
          13       0.00      0.00      0.00       192
          14       0.00      0.00      0.00        88
          15       0.00      0.00      0.00       

/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [19]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x1a182ae200>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('mcfl',
   MultiOutputClassifier(estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
     kernel='rbf', max_iter=-1, probability=False, random_state=None,
     shrinking=True, tol=0.001, verbose=False),
              n_jobs=None))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, en

In [20]:
from sklearn.svm import LinearSVC
X = df.iloc[:, 1]
Y = df.iloc[:, 4:].drop(columns='child_alone')

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('mcfl', MultiOutputClassifier(estimator=LinearSVC()))
])

parameters = {
    'mcfl__estimator__C' : [1],
    'mcfl__estimator__loss' : ['hinge', 'squared_hinge'],
    'mcfl__estimator__max_iter' : [1000, 10000]
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10, cv=3, n_jobs=12, scoring='f1_micro')

cv.fit(X_train, Y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:   38.4s
[Parallel(n_jobs=12)]: Done   3 out of  12 | elapsed:   39.3s remaining:  2.0min
[Parallel(n_jobs=12)]: Done   5 out of  12 | elapsed:   42.2s remaining:   59.1s
[Parallel(n_jobs=12)]: Done   7 out of  12 | elapsed:   42.3s remaining:   30.2s
[Parallel(n_jobs=12)]: Done   9 out of  12 | elapsed:   42.3s remaining:   14.1s
[Parallel(n_jobs=12)]: Done  12 out of  12 | elapsed:   43.9s finished
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=12,
       param_grid={'mcfl__estimator__C': [1], 'mcfl__estimator__loss': ['hinge', 'squared_hinge'], 'mcfl__estimator__max_iter': [1000, 10000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=10)

In [21]:
Y_pred = cv.predict(X_test)
print(cv.best_params_)
print(cv.best_score_)
print(classification_report(Y_test, Y_pred))

{'mcfl__estimator__C': 1, 'mcfl__estimator__loss': 'hinge', 'mcfl__estimator__max_iter': 1000}
0.6772156709563802
              precision    recall  f1-score   support

           0       0.86      0.93      0.89      5941
           1       0.78      0.58      0.67      1333
           2       0.00      0.00      0.00        34
           3       0.75      0.69      0.72      3286
           4       0.65      0.20      0.31       644
           5       0.75      0.26      0.39       414
           6       0.79      0.13      0.22       239
           7       0.00      0.00      0.00       156
           8       0.67      0.25      0.37       267
           9       0.75      0.66      0.70       512
          10       0.79      0.76      0.77       878
          11       0.80      0.55      0.65       714
          12       0.76      0.45      0.56       123
          13       0.81      0.13      0.22       192
          14       0.57      0.18      0.28        88
          15       0.

/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [22]:
cv.get_params()

{'cv': 3,
 'error_score': 'raise-deprecating',
 'estimator__memory': None,
 'estimator__steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x1a182ae200>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('mcfl',
   MultiOutputClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
        intercept_scaling=1, loss='squared_hinge', max_iter=1000,
        multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
        verbose=0),
              n_jobs=None))],
 'estimator__vect': CountVectorizer(analyzer='word', binary=False, deco

In [23]:
from sklearn.naive_bayes import MultinomialNB
X = df.iloc[:, 1]
Y = df.iloc[:, 4:] #.drop(columns='child_alone')

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('mcfl', MultiOutputClassifier(estimator=MultinomialNB()))
])

parameters = {
    'mcfl__estimator__alpha' : [0.01, 0.1, 1, 10]
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10, cv=3, n_jobs=12, scoring='f1_micro')

cv.fit(X_train, Y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:   41.1s
[Parallel(n_jobs=12)]: Done   3 out of  12 | elapsed:   41.2s remaining:  2.1min
[Parallel(n_jobs=12)]: Done   5 out of  12 | elapsed:   41.4s remaining:   57.9s
[Parallel(n_jobs=12)]: Done   7 out of  12 | elapsed:   41.5s remaining:   29.6s
[Parallel(n_jobs=12)]: Done   9 out of  12 | elapsed:   41.5s remaining:   13.8s
[Parallel(n_jobs=12)]: Done  12 out of  12 | elapsed:   41.7s finished
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/naive_bayes.py:465: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...fier(estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=12,
       param_grid={'mcfl__estimator__alpha': [0.01, 0.1, 1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=10)

In [24]:
Y_pred = cv.predict(X_test)
print(cv.best_params_)
print(cv.best_score_)
print(classification_report(Y_test, Y_pred))

{'mcfl__estimator__alpha': 0.01}
0.587659155515815
              precision    recall  f1-score   support

           0       0.83      0.94      0.88      5941
           1       0.67      0.59      0.63      1333
           2       0.00      0.00      0.00        34
           3       0.67      0.66      0.66      3286
           4       0.54      0.21      0.30       644
           5       0.49      0.22      0.30       414
           6       0.58      0.06      0.11       239
           7       0.17      0.01      0.02       156
           8       0.55      0.30      0.39       267
           9       0.00      0.00      0.00         0
          10       0.51      0.20      0.28       512
          11       0.61      0.37      0.46       878
          12       0.59      0.27      0.37       714
          13       0.48      0.24      0.32       123
          14       0.51      0.10      0.17       192
          15       0.25      0.02      0.04        88
          16       0.23      0

/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/stingl/anaconda3/envs/nd025_aktuell/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being se

In [25]:
cv.get_params()

{'cv': 3,
 'error_score': 'raise-deprecating',
 'estimator__memory': None,
 'estimator__steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x1a182ae200>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('mcfl',
   MultiOutputClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
              n_jobs=None))],
 'estimator__vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
       

### 9. Export your model as a pickle file

In [26]:
joblib.dump(cv.best_estimator_, 'classifier.pkl', compress = 1)

['classifier.pkl']

In [27]:
cls = joblib.load('classifier.pkl')
pd.DataFrame(cls.predict(X_test)).head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
4,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.